In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import time

from datetime import timedelta
import datetime

In [2]:
df = pd.read_csv("./min_kospi200F(20200909).csv", index_col=0)
df

,date,time,open,high,low,close,prevClose,vol
0,20100216,901,207.55,207.65,207.50,207.60,207.50,3985
1,20100216,902,207.60,207.65,207.25,207.55,207.50,5095
2,20100216,903,207.55,207.80,207.50,207.60,207.50,2175
3,20100216,904,207.55,207.85,207.55,207.80,207.50,1301
4,20100216,905,207.80,208.15,207.80,208.05,207.50,3870
...,...,...,...,...,...,...,...,...
941511,20200403,1534,231.40,231.40,231.30,231.30,231.65,210
941512,20200403,1535,231.30,231.50,231.30,231.40,231.65,932
941513,20200403,1535,231.30,231.50,231.30,231.40,231.65,932
941514,20200403,1545,231.65,231.65,231.65,231.65,231.65,6748


# 중복 데이터 삭제

In [3]:
df.duplicated().sum() # 중복 데이터 존재

np.int64(396)

In [4]:
df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
print("중복 데이터 개수:", df.duplicated().sum())
print("전체 데이터 개수:", len(df))

중복 데이터 개수: 0
전체 데이터 개수: 941120


In [5]:
df.isna().sum()

date         0
time         0
open         0
high         0
low          0
close        0
prevClose    0
vol          0
dtype: int64

# datetime 생성 및 index 지정

In [6]:
# date, time을 Timestamp 형식으로 변환
df['date'] = df['date'].astype(str)
df['time'] = df['time'].astype(str)
date_time = pd.to_datetime(df['date'] + df['time'], format='%Y%m%d%H%M')

# datetime을 index로 사용
df.index = date_time

# date, time 열 삭제
# df.drop(['date', 'time'], axis=1, inplace=True)
df

,date,time,open,high,low,close,prevClose,vol
2010-02-16 09:01:00,20100216,901,207.55,207.65,207.50,207.60,207.50,3985
2010-02-16 09:02:00,20100216,902,207.60,207.65,207.25,207.55,207.50,5095
2010-02-16 09:03:00,20100216,903,207.55,207.80,207.50,207.60,207.50,2175
2010-02-16 09:04:00,20100216,904,207.55,207.85,207.55,207.80,207.50,1301
2010-02-16 09:05:00,20100216,905,207.80,208.15,207.80,208.05,207.50,3870
...,...,...,...,...,...,...,...,...
2020-04-03 15:32:00,20200403,1532,231.40,231.40,231.30,231.35,231.65,876
2020-04-03 15:33:00,20200403,1533,231.35,231.40,231.30,231.35,231.65,586
2020-04-03 15:34:00,20200403,1534,231.40,231.40,231.30,231.30,231.65,210
2020-04-03 15:35:00,20200403,1535,231.30,231.50,231.30,231.40,231.65,932


# 장이 늦게 열린 날

- 수능 날에는 장이 1시간 늦게 열리고 1시간 늦게 마감한다.

In [7]:
data_latetime = df[(df.index.time >= pd.to_datetime('16:00:00').time())]
late_time_date = data_latetime['date'].unique()
late_time_date

array(['20101118', '20111110', '20121108', '20131107', '20141113',
       '20151112', '20161117', '20171116', '20171123', '20181115',
       '20191114'], dtype=object)

# 장 마감 10분 전, 15:06 / 16:06 데이터 처리

In [8]:
data_1506 = df[(df.index.time == pd.to_datetime('15:06:00').time()) & (df.index.date < pd.to_datetime('2016-08-01').date())]

for idx in data_1506.index:
    if data_1506['date'][idx] in late_time_date: # 수능날은 장이 닫히는 시간이 다르므로 제외
        data_1506.drop(idx, axis=0, inplace=True)
    else:
        # 15:05 데이터의 close값만 15:06 데이터의 값으로 변경하고 15:06 데이터 삭제
        df[df.index == idx-timedelta(minutes=1)]['close'] = data_1506['close'][idx]
        df[df.index == idx-timedelta(minutes=1)]['vol'] += data_1506['vol'][idx]
        df.drop(idx, axis=0, inplace=True)

/var/folders/pz/1m8kjyh1283_znnjwmd3h7740000gn/T/ipykernel_8153/3301688109.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.index == idx-timedelta(minutes=1)]['close'] = data_1506['close'][idx]
/var/folders/pz/1m8kjyh1283_znnjwmd3h7740000gn/T/ipykernel_8153/3301688109.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a Data

In [10]:
# 제대로 처리되었는지 확인
df[(df.index.time == pd.to_datetime('15:06:00').time()) & (df.index.date < pd.to_datetime('2016-08-01').date())]

,date,time,open,high,low,close,prevClose,vol
2010-11-18 15:06:00,20101118,1506,251.00,251.15,251.00,251.10,247.45,395
2011-11-10 15:06:00,20111110,1506,240.40,240.45,240.30,240.40,250.60,601
2012-11-08 15:06:00,20121108,1506,249.90,250.10,249.85,250.05,254.05,769
2013-11-07 15:06:00,20131107,1506,263.70,263.80,263.65,263.65,265.60,402
2014-11-13 15:06:00,20141113,1506,250.65,250.70,250.60,250.65,251.45,281
2015-11-12 15:06:00,20151112,1506,245.95,246.00,245.95,246.00,246.00,133


In [11]:
# 수능 날, 장 마감 전 16:06 데이터 존재 확인
df[(df.index.time == pd.to_datetime('16:06:00').time()) & (df.index.date < pd.to_datetime('2016-08-01').date())]

,date,time,open,high,low,close,prevClose,vol
2010-11-18 16:06:00,20101118,1606,251.6,251.6,251.6,251.6,247.45,1


In [12]:
idx = df[(df.index.time == pd.to_datetime('16:06:00').time()) & (df.index.date < pd.to_datetime('2016-08-01').date())].index[0]
df[df.index == idx-timedelta(minutes=1)]['close'] = df[(df.index.time == pd.to_datetime('16:06:00').time()) & (df.index.date < pd.to_datetime('2016-08-01').date())]['close']
df.drop(idx, axis=0, inplace=True)

/var/folders/pz/1m8kjyh1283_znnjwmd3h7740000gn/T/ipykernel_8153/922659267.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.index == idx-timedelta(minutes=1)]['close'] = df[(df.index.time == pd.to_datetime('16:06:00').time()) & (df.index.date < pd.to_datetime('2016-08-01').date())]['close']


In [13]:
# 처리된 것 확인
df[(df.index.time == pd.to_datetime('16:06:00').time()) & (df.index.date < pd.to_datetime('2016-08-01').date())]

,date,time,open,high,low,close,prevClose,vol


# 순수 결측치 처리

In [14]:
np.sort(df.groupby('date')['time'].count().unique())

array([  1, 306, 331, 336, 361, 362, 363, 364, 365, 366, 367, 387, 390,
       391, 393, 394, 395, 396])

## 2010-02-16 : 데이터 1개만 존재 -> 그냥 결측치

In [15]:
np.where(df.groupby('date')['time'].count() == 1)

(array([104]),)

In [16]:
df.groupby('date')['time'].count().index[104]

'20100716'

In [17]:
df[df['date'] == '20100716']

,date,time,open,high,low,close,prevClose,vol
2010-07-16 15:15:00,20100716,1515,227.05,229.95,227.05,229.1,229.65,3901


## 타임스탬프 생성 및 결측치 Nan으로 패딩
서킷 브레이크 30분도 Nan처리, 매일 장 마감 10분 전 블라인드 타임은 타임스탬프를 생성하지 않음.

In [18]:
strange_date = []
new_year_date = []
sat_date = []
for day, group in df.groupby(df.index.date):
    if datetime.time(10, 0) not in group.index.time:
        strange_date.append(day)

        if day.month == 1:
            new_year_date.append(day)
        
        if day.month == 11:
            sat_date.append(day)

print(strange_date)
print(new_year_date)
print(sat_date)

[datetime.date(2010, 7, 16), datetime.date(2010, 11, 18), datetime.date(2011, 1, 3), datetime.date(2011, 11, 10), datetime.date(2012, 1, 2), datetime.date(2012, 11, 8), datetime.date(2013, 1, 2), datetime.date(2013, 11, 7), datetime.date(2014, 1, 2), datetime.date(2014, 11, 13), datetime.date(2015, 1, 2), datetime.date(2015, 11, 12), datetime.date(2016, 1, 4), datetime.date(2016, 11, 17), datetime.date(2017, 1, 2), datetime.date(2017, 11, 16), datetime.date(2017, 11, 23), datetime.date(2018, 1, 2), datetime.date(2018, 11, 15), datetime.date(2019, 1, 2), datetime.date(2019, 11, 14), datetime.date(2020, 1, 2)]
[datetime.date(2011, 1, 3), datetime.date(2012, 1, 2), datetime.date(2013, 1, 2), datetime.date(2014, 1, 2), datetime.date(2015, 1, 2), datetime.date(2016, 1, 4), datetime.date(2017, 1, 2), datetime.date(2018, 1, 2), datetime.date(2019, 1, 2), datetime.date(2020, 1, 2)]
[datetime.date(2010, 11, 18), datetime.date(2011, 11, 10), datetime.date(2012, 11, 8), datetime.date(2013, 11, 7)

In [19]:
def make_time_range(day, open_h, open_m, close_h, close_m):
    start_time = datetime.datetime.combine(day, datetime.time(open_h, open_m))  # day와 시간 결합
    end_time = datetime.datetime.combine(day, datetime.time(close_h, close_m-10))  # day와 시간 결합
    time_range = pd.date_range(start=start_time, end=end_time, freq='1T')

    close_time = datetime.datetime.combine(day, datetime.time(close_h, close_m))
    time_range = time_range.append(pd.DatetimeIndex([close_time]))

    return time_range

In [20]:
full_time_range = []
for day, group in df.groupby(df.index.date):
    if day < datetime.datetime(2016, 8, 1).date():
        if day in sat_date: # 수능 날 
            time_range = make_time_range(day, 10, 1, 16, 15)

        elif day in new_year_date: # 신년
            time_range = make_time_range(day, 10, 1, 15, 15)

        else:
            time_range = make_time_range(day, 9, 1, 15, 15)

    else:
        if day in sat_date: # 수능 날 
            time_range = make_time_range(day, 10, 1, 16, 45)

        elif day in new_year_date: # 신년
            time_range = make_time_range(day, 10, 1, 15, 45)

        else:
            time_range = make_time_range(day, 9, 1, 15, 45)

    full_time_range.append(time_range)

combined_time_range = pd.concat([pd.Series(tr) for tr in full_time_range])
df = df.reindex(combined_time_range)

/var/folders/pz/1m8kjyh1283_znnjwmd3h7740000gn/T/ipykernel_8153/142564003.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_range = pd.date_range(start=start_time, end=end_time, freq='1T')


In [21]:
df

,date,time,open,high,low,close,prevClose,vol
2010-02-16 09:01:00,20100216,901,207.55,207.65,207.50,207.60,207.50,3985.0
2010-02-16 09:02:00,20100216,902,207.60,207.65,207.25,207.55,207.50,5095.0
2010-02-16 09:03:00,20100216,903,207.55,207.80,207.50,207.60,207.50,2175.0
2010-02-16 09:04:00,20100216,904,207.55,207.85,207.55,207.80,207.50,1301.0
2010-02-16 09:05:00,20100216,905,207.80,208.15,207.80,208.05,207.50,3870.0
...,...,...,...,...,...,...,...,...
2020-04-03 15:32:00,20200403,1532,231.40,231.40,231.30,231.35,231.65,876.0
2020-04-03 15:33:00,20200403,1533,231.35,231.40,231.30,231.35,231.65,586.0
2020-04-03 15:34:00,20200403,1534,231.40,231.40,231.30,231.30,231.65,210.0
2020-04-03 15:35:00,20200403,1535,231.30,231.50,231.30,231.40,231.65,932.0


In [22]:
df.isna().sum()

date         829
time         829
open         829
high         829
low          829
close        829
prevClose    829
vol          829
dtype: int64

In [23]:
df.to_pickle("kospi200_preprocessed.pkl")

In [24]:
df.index

DatetimeIndex(['2010-02-16 09:01:00', '2010-02-16 09:02:00',
               '2010-02-16 09:03:00', '2010-02-16 09:04:00',
               '2010-02-16 09:05:00', '2010-02-16 09:06:00',
               '2010-02-16 09:07:00', '2010-02-16 09:08:00',
               '2010-02-16 09:09:00', '2010-02-16 09:10:00',
               ...
               '2020-04-03 15:27:00', '2020-04-03 15:28:00',
               '2020-04-03 15:29:00', '2020-04-03 15:30:00',
               '2020-04-03 15:31:00', '2020-04-03 15:32:00',
               '2020-04-03 15:33:00', '2020-04-03 15:34:00',
               '2020-04-03 15:35:00', '2020-04-03 15:45:00'],
              dtype='datetime64[ns]', length=941826, freq=None)

In [25]:
print(df[df.isna().any(axis=1)])

                    date time  open  high  low  close  prevClose  vol
2010-07-16 09:01:00  NaN  NaN   NaN   NaN  NaN    NaN        NaN  NaN
2010-07-16 09:02:00  NaN  NaN   NaN   NaN  NaN    NaN        NaN  NaN
2010-07-16 09:03:00  NaN  NaN   NaN   NaN  NaN    NaN        NaN  NaN
2010-07-16 09:04:00  NaN  NaN   NaN   NaN  NaN    NaN        NaN  NaN
2010-07-16 09:05:00  NaN  NaN   NaN   NaN  NaN    NaN        NaN  NaN
...                  ...  ...   ...   ...  ...    ...        ...  ...
2020-03-19 12:31:00  NaN  NaN   NaN   NaN  NaN    NaN        NaN  NaN
2020-03-19 12:32:00  NaN  NaN   NaN   NaN  NaN    NaN        NaN  NaN
2020-03-19 12:33:00  NaN  NaN   NaN   NaN  NaN    NaN        NaN  NaN
2020-03-19 12:34:00  NaN  NaN   NaN   NaN  NaN    NaN        NaN  NaN
2020-03-19 12:35:00  NaN  NaN   NaN   NaN  NaN    NaN        NaN  NaN

[829 rows x 8 columns]
